<a href="https://colab.research.google.com/github/DGuilherme/BasicDT/blob/master/CH3_PatentRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge 3 Patent Reconmmender System


In [2]:
# Import section
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

# solve issue of gensim version
!pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


# Import Dataset

*Dataset composition*   

| Feature        | Description           |
| -------------- | --------------------- |
| ID             | The patent ID         |
| Title          | Patent Title          |
| Abstract       | Patent Abstract       |
| Classification | [Patent Classification](https://www.uspto.gov/web/patents/classification/selectnumwithtitle.htm) |

## How to export from database


```
db.10000.find(
  {'classes.FSC': {$exists: true},title: {$exists: true},'abstract': {$exists: true}},
  {'abstract': 1,title: 1,'classes.FSC': 1}
)
```



In [3]:
url = 'https://raw.githubusercontent.com/DGuilherme/Challenge3/main/Dataset/10000_classified_patents.json'


# Preprocessing 


In [ ]:
from sklearn.model_selection import train_test_split

train_data_unsplit = pd.read_json(url)
train_data_unsplit = train_data_unsplit.rename(columns={'_id': 'ID', 'abstract': 'Resumo','title': 'Titulo'})
train_data_unsplit = train_data_unsplit.dropna()
train_data_unsplit = train_data_unsplit.drop_duplicates(subset ="Resumo",keep = False)
train_data_unsplit = train_data_unsplit.drop_duplicates(subset ="Titulo",keep = False)

# Split dataset
train_data, test_data = train_test_split(train_data_unsplit, test_size=0.2)

# Create the Vocabulary

In [ ]:
import gensim
def tagData(dataframe):
  number = 0
  for index,row in dataframe.iterrows():
    number = number + 1
    resumotokens = gensim.utils.simple_preprocess(row['Resumo'])
    #titulotokens = gensim.utils.simple_preprocess(row['Titulo'])

    yield gensim.models.doc2vec.TaggedDocument(resumotokens, [number]) #yield gensim.utils.simple_preprocess(row['Resumo'])


vocabulary = list(tagData(train_data))
vocabulary_test = list(tagData(test_data))

# User Question


# Create gensim Doc2Vec model

In [ ]:
# instanciate
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=250) # Create inital empty model

# build
model.build_vocab(vocabulary) # Add data to the model

# Model Train


In [ ]:
model.train(vocabulary, total_examples=model.corpus_count, epochs=model.epochs)

# Model Test
 